In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
import csv

In [2]:
data_fake=pd.read_csv('Fake.csv', on_bad_lines='skip')
data_true=pd.read_csv('True.csv', on_bad_lines='skip')

In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
data_fake['class']=0
data_true['class']=1

In [6]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_merge=pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [8]:
data=data_merge.drop(['title','subject','date'],axis=1)

In [9]:
data.isnull().sum()

,0
text,0
class,0


In [10]:
data=data.sample(frac=1)

In [11]:
data.head()

,text,class
5008,Ivanka Trump enjoys what someone from that fam...,0
13462,Trump talked about a country on the brink of d...,0
17745,Paul Joseph Watson nails it again. In the vide...,0
14459,DHAKA (Reuters) - Chinese Foreign Minister Wan...,1
5082,WASHINGTON (Reuters) - The U.S. Senate on Tues...,1


In [12]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

In [13]:
data.columns

Index(['text', 'class'], dtype='object')

In [14]:
def wordopt(text):
  text=text.lower()
  text=re.sub('\[.*?\]','',text)
  text=re.sub("\\W"," ",text)
  text=re.sub('https?://\S+|www\.\S+','',text)
  text=re.sub('<.*?>+','',text)
  text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
  text= re.sub('\n','',text)
  text=re.sub('\w*\d\w*','',text)
  return text

In [15]:
data['text']=data['text'].apply(wordopt)

In [16]:
x=data['text']
y=data['class']

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

In [19]:
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [20]:
pred_lr=LR.predict(xv_test)

In [21]:
LR.score(xv_test,y_test)

0.9860801781737194

In [22]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4650
           1       0.98      0.99      0.99      4330

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [23]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [24]:
pred_dt=DT.predict(xv_test)

In [25]:
print(classification_report(y_test,pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      4650
           1       1.00      0.99      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [26]:
from sklearn.ensemble import GradientBoostingClassifier
GB=GradientBoostingClassifier(random_state=0)
GB.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [27]:
predict_gb=GB.predict(xv_test)

In [29]:
print(classification_report(y_test,predict_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4650
           1       0.99      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [30]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier(random_state=0)
RF.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [31]:
pred_rf=RF.predict(xv_test)

In [32]:
print(classification_report(y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4650
           1       0.99      0.99      0.99      4330

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [38]:
def output_label(n):
  if n==0:
    return 'Fake News'
  elif n==1:
    return 'Not a fake news'
def manual_testing(news):
  testing_news={'text':[news]}
  new_def_test=pd.DataFrame(testing_news)
  new_def_test['text']=new_def_test['text'].apply(wordopt)
  new_x_test=new_def_test['text']
  new_xv_test=vectorization.transform(new_x_test)
  pred_LR=LR.predict(new_xv_test)
  pred_DT=DT.predict(new_xv_test)
  pred_GB=GB.predict(new_xv_test)
  pred_RF=RF.predict(new_xv_test)
  return print(f"\n\nLR Prediction: {output_label(pred_LR[0])} \n" f"DT Prediction: {output_label(pred_DT[0])} \n" f"GBC Prediction: {output_label(pred_GB[0])} \n" f"RFC Prediction: {output_label(pred_RF[0])}")

In [39]:
news=str(input())
manual_testing(news)

President Trump and Chinese leader Xi Jinping spoke on Thursday, and both countries pledged to restart tariff and trade talks in the coming days.  Trump hailed the call as "positive," with both leaders inviting the other to visit their respective countries. Chinese state media said Xi urged Trump to remove "negative" trade measures on his country.


LR Prediction: Not a fake news 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Not a fake news


In [40]:
news=str(input())
manual_testing(news)

Days after leaving his role in the White House, Elon Musk excoriated President Donald Trump's tax and domestic policy bill as a "disgusting abomination," escalating his criticism of the president's signature legislation over concerns that it will increase the deficit.  Musk's pointed June 3 remarks came after Trump held a friendly send-off for the world's richest man in the Oval Office last Friday and as the second-term Republican president is working to push what he's called his "big, beautiful bill" through the Senate.


LR Prediction: Not a fake news 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Not a fake news
